### [Building Fallbacks to Websearch with Conditional Routing](https://haystack.deepset.ai/tutorials/36_building_fallbacks_with_conditional_routing)

In [1]:
#! Solo es para haystack sepa que tutorial se esta ejecutando
from haystack.telemetry import tutorial_running
tutorial_running(36)

### 1. Código Original

In [2]:
from getpass import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
if "SERPERDEV_API_KEY" not in os.environ:
    os.environ["SERPERDEV_API_KEY"] = getpass("Enter Serper Api key: ")

In [3]:
from haystack.dataclasses import Document

documents = [
    Document(
        content="""Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural
                                heritage and modern urban sophistication. Nestled along the banks of the Isar River, Munich is renowned
                                for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and
                                the grandeur of Nymphenburg Palace. The city is a haven for art enthusiasts, with world-class museums like the
                                Alte Pinakothek housing masterpieces by renowned artists. Munich is also famous for its lively beer gardens, where
                                locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine. The city's annual
                                Oktoberfest celebration, the world's largest beer festival, attracts millions of visitors from around the globe.
                                Beyond its cultural and culinary delights, Munich offers picturesque parks like the English Garden, providing a
                                serene escape within the heart of the bustling metropolis. Visitors are charmed by Munich's warm hospitality,
                                making it a must-visit destination for travelers seeking a taste of both old-world charm and contemporary allure."""
    )
]


In [4]:
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

prompt_template = [ChatMessage.from_user("""
Answer the following query given the documents.
If the answer is not contained within the documents reply with 'no_answer'
Query: {{query}}
Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}
""")]

prompt_builder = ChatPromptBuilder(template=prompt_template)
llm = OpenAIChatGenerator(model="gpt-4o-mini")


In [5]:

from haystack.components.websearch.serper_dev import SerperDevWebSearch

prompt_for_websearch = [ChatMessage.from_user("""
Answer the following query given the documents retrieved from the web.
Your answer shoud indicate that your answer was generated from websearch.

Query: {{query}}
Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}
""")]

websearch = SerperDevWebSearch()
prompt_builder_for_websearch = ChatPromptBuilder(template=prompt_for_websearch)
llm_for_websearch = OpenAIChatGenerator(model="gpt-4o-mini")


In [6]:
from haystack.components.routers import ConditionalRouter

routes = [
    {
        "condition": "{{'no_answer' in replies[0].text}}",
        "output": "{{query}}",
        "output_name": "go_to_websearch",
        "output_type": str,
    },
    {
        "condition": "{{'no_answer' not in replies[0].text}}",
        "output": "{{replies[0].text}}",
        "output_name": "answer",
        "output_type": str,
    },
]

router = ConditionalRouter(routes)


In [7]:
from haystack import Pipeline

pipe = Pipeline()
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.add_component("router", router)
pipe.add_component("websearch", websearch)
pipe.add_component("prompt_builder_for_websearch", prompt_builder_for_websearch)
pipe.add_component("llm_for_websearch", llm_for_websearch)

pipe.connect("prompt_builder.prompt", "llm.messages")
pipe.connect("llm.replies", "router.replies")
pipe.connect("router.go_to_websearch", "websearch.query")
pipe.connect("router.go_to_websearch", "prompt_builder_for_websearch.query")
pipe.connect("websearch.documents", "prompt_builder_for_websearch.documents")
pipe.connect("prompt_builder_for_websearch", "llm_for_websearch")


🚅 Components
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
  - router: ConditionalRouter
  - websearch: SerperDevWebSearch
  - prompt_builder_for_websearch: ChatPromptBuilder
  - llm_for_websearch: OpenAIChatGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])
  - llm.replies -> router.replies (List[ChatMessage])
  - router.go_to_websearch -> websearch.query (str)
  - router.go_to_websearch -> prompt_builder_for_websearch.query (str)
  - websearch.documents -> prompt_builder_for_websearch.documents (List[Document])
  - prompt_builder_for_websearch.prompt -> llm_for_websearch.messages (List[ChatMessage])

In [8]:
query = "Where is Munich?"

result = pipe.run({"prompt_builder": {"query": query, "documents": documents}, "router": {"query": query}})

# Print the `answer` coming from the ConditionalRouter
result


{'router': {'answer': ''}}

In [9]:
query = "How many people live in Munich?"

result = pipe.run({"prompt_builder": {"query": query, "documents": documents}, "router": {"query": query}})

# Print the `replies` generated using the web searched Documents
result


{'router': {'answer': ''}}

### 2. Depuración de código

In [10]:
from haystack import Document
from haystack.components.preprocessors import DocumentSplitter

splitter = DocumentSplitter(split_length=1, split_overlap=0, split_by="sentence")

In [14]:
text = """
Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural heritage and modern urban sophistication. Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and the grandeur of Nymphenburg Palace. The city is a haven for art enthusiasts, with world-class museums like the Alte Pinakothek housing masterpieces by renowned artists. Munich is also famous for its lively beer gardens, where locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine. The city's annual Oktoberfest celebration, the world's largest beer festival, attracts millions of visitors from around the globe. Beyond its cultural and culinary delights, Munich offers picturesque parks like the English Garden, providing a serene escape within the heart of the bustling metropolis. Visitors are charmed by Munich's warm hospitality, making it a must-visit destination for travelers seeking a taste of both old-world charm and contemporary allure.
"""

doc = Document(content=text)
docs = splitter.run([doc])
docs

{'documents': [Document(id=1bff9e298ad99e305006ab2d4693a9a65c0cbdc862379fe0d3c947fabf1663d8, content: '
  Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural...', meta: {'source_id': '35490afe62ba66d8906384ec746fc6bec2ba44541c27f36089378a7f6c8d5623', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}),
  Document(id=92b88976af56b006e1d3ecf7c6a15926549f8fdbd91d9d076b6ded74f8fce530, content: ' Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, includ...', meta: {'source_id': '35490afe62ba66d8906384ec746fc6bec2ba44541c27f36089378a7f6c8d5623', 'page_number': 1, 'split_id': 1, 'split_idx_start': 142}),
  Document(id=d6c7eee36337df76d578c984d2823abb8a1c966505373c8e11f9351493adbef4, content: ' The city is a haven for art enthusiasts, with world-class museums like the Alte Pinakothek housing ...', meta: {'source_id': '35490afe62ba66d8906384ec746fc6bec2ba44541c27f36089378a7f6c8d5623', 'page_number': 1,

In [17]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy

doc_store = InMemoryDocumentStore()
doc_store.write_documents(docs['documents'], policy=DuplicatePolicy.OVERWRITE)
doc_store.filter_documents()

[Document(id=1bff9e298ad99e305006ab2d4693a9a65c0cbdc862379fe0d3c947fabf1663d8, content: '
 Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural...', meta: {'source_id': '35490afe62ba66d8906384ec746fc6bec2ba44541c27f36089378a7f6c8d5623', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}),
 Document(id=92b88976af56b006e1d3ecf7c6a15926549f8fdbd91d9d076b6ded74f8fce530, content: ' Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, includ...', meta: {'source_id': '35490afe62ba66d8906384ec746fc6bec2ba44541c27f36089378a7f6c8d5623', 'page_number': 1, 'split_id': 1, 'split_idx_start': 142}),
 Document(id=d6c7eee36337df76d578c984d2823abb8a1c966505373c8e11f9351493adbef4, content: ' The city is a haven for art enthusiasts, with world-class museums like the Alte Pinakothek housing ...', meta: {'source_id': '35490afe62ba66d8906384ec746fc6bec2ba44541c27f36089378a7f6c8d5623', 'page_number': 1, 'split_id': 2, '

In [18]:
from haystack.components.retrievers import SentenceWindowRetriever

retriever = SentenceWindowRetriever(document_store=doc_store, window_size=2)
result = retriever.run(retrieved_documents=[docs['documents'][2]])
result

{'context_windows': ["\nMunich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural heritage and modern urban sophistication. Nestled along the banks of the Isar River, Munich is renowned for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and the grandeur of Nymphenburg Palace. The city is a haven for art enthusiasts, with world-class museums like the Alte Pinakothek housing masterpieces by renowned artists. Munich is also famous for its lively beer gardens, where locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine. The city's annual Oktoberfest celebration, the world's largest beer festival, attracts millions of visitors from around the globe."],
 'context_documents': [[Document(id=1bff9e298ad99e305006ab2d4693a9a65c0cbdc862379fe0d3c947fabf1663d8, content: '
   Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural...